In [10]:
import os
import pandas as pd
from matplotlib.pyplot import plot

In [7]:
MAIN_PATH = 'D:/Master of Applied IT' 
DATA_SUBPATH = '/data'
UNPROCESSED_PATH = MAIN_PATH + DATA_SUBPATH + '/unprocessed/WESAD'
PROCESSED_PATH = MAIN_PATH + DATA_SUBPATH + '/processed/WESAD'

In [22]:
final_columns = {
    'ACC': ['id','X', 'Y', 'datetime'],
    'EDA': ['id','EDA','datetime'],
    'HR': ['id','HR','datetime'],
    'TEMP': ['id','TEMP','datetime']
}

names = {
    'ACC.csv': ['X', 'Y', 'Z'],
    'EDA.csv': ['EDA'],
    'HR.csv': ['HR'],
    'TEMP.csv': ['TEMP']
}

desired_signals = ['ACC.csv', 'EDA.csv', 'HR.csv', 'TEMP.csv']
acc = pd.DataFrame(columns=final_columns['ACC'])
eda = pd.DataFrame(columns=final_columns['EDA'])
hr = pd.DataFrame(columns=final_columns['HR'])
temp = pd.DataFrame(columns=final_columns['TEMP'])

In [ ]:
def process_df(df,file):
    start_timestamp = df.iloc